# 人間データの生成

In [30]:
!@cd

d:\Source\repos\ds_bias\neosim2\preprocessing


In [31]:
import numpy as np
import polars as pl
import pandas as pd

In [32]:
import sys
sys.path.append(f"../../neosim/exp")

In [33]:
import json

In [34]:
from dataset_preprocessing import change_r

In [35]:
np.random.seed(777)

### Tlk Agg Ftr2

In [36]:
df = pd.read_csv("../../datasets/crowd-kit-format/tlkaggftrs.csv", dtype={"task" : str, "worker" :str ,"label" : str ,"gt" : str})
gt = df.filter(["task","gt"]).drop_duplicates(keep='last')
df = df.drop(["gt"], axis=1)

n_classes = 5
r_range = [3,5]
labels = [0,1,2,3,4]
name = "adult"

In [37]:
df["label"] = (df["label"].astype(int) - 1).astype(str)
gt["gt"] = (gt["gt"].astype(int) - 1).astype(str)

In [38]:
df = pl.from_pandas(df)
gt = pl.from_pandas(gt)

In [39]:
tab = gt.group_by("gt").count().select("gt", (pl.col("count") / gt.select(pl.count())).alias("ratio")).sort("gt")
tab

C:\Users\tamut\AppData\Local\Temp\ipykernel_22324\1498582118.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  tab = gt.group_by("gt").count().select("gt", (pl.col("count") / gt.select(pl.count())).alias("ratio")).sort("gt")
C:\Users\tamut\AppData\Local\Temp\ipykernel_22324\1498582118.py:1: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  tab = gt.group_by("gt").count().select("gt", (pl.col("count") / gt.select(pl.count())).alias("ratio")).sort("gt")


gt,ratio
str,f64
"""0""",0.061554
"""1""",0.041372
"""2""",0.551968
"""3""",0.055499
"""4""",0.289606


In [40]:
raw_df_pl = df

In [41]:
def write_data(name, raw_df_pl, r_range, gt):
    for r in r_range:
        df = change_r(raw_df_pl, r)
        df.write_csv(f"../human_responses/{name}_r={r}.csv")
    gt.write_csv(f"../human_responses/{name}_gt.csv")

In [42]:
write_data(name, raw_df_pl, r_range, gt)

Now Reducdancy: 60.614530776992936
New Reducdancy: 3.0
Now Reducdancy: 60.614530776992936
New Reducdancy: 5.0


In [43]:
## メモ 10はたりないみたい

In [44]:
def save_dataset_profile(name, r_range, n_classes, labels):
    data = {}
    data["dataset_name"] = name
    data["r_range"] = r_range
    data["n_classes"] = n_classes
    data["labels"] = labels
    json.dump(data, open(f"../human_responses/{name}_dataset_profile.json", "w"), indent=4)

In [45]:
save_dataset_profile(name, r_range, n_classes, labels)